### Ejercicio 1.
Escribir un programa que cree un número de threads indicado por el primer argumento, de forma que:
- Cada thread se le asignará un identificador 0,1,2…  que imprimirá por la salida estándar y usará para hacer un sleep(3) de los mismos segundos.
- El thread principal esperará a que terminen todos los threads, mostrando el identificador del thread que termina.


In [ ]:
#include <stdio.h>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>

// Definicion de la estructura para pasar argumentos a los threads.
typedef struct _thread_info_t{ // typedef sirve para definir un nuevo tipo de dato.
    int num_thread; // Numero de thread.
    pthread_t id; // Identificador del thread a nivel de POSIX.
} thread_info;

void* haz_thread(void* _info) {
    thread_info_t* info = (thread_info_t*) _info; // casteamos el puntero void a thread_info_t.

    printf("Thread %i, sleeping...\n", info->num_thread);
    sleep(info->num_thread); //indica el numero de thread

    return NULL;
}

int main(int argc, char *argv[]) {
    // Primer argumento es el numero de threads.
    int nt = atoi(argv[1]);

    // reservamos memoria para los threads "malloc(nt * sizeof(thread_info_t))".
    // Creamos un array de estructuras thread_info_t.
    thread_info_t * pool = (thread_info_t*) malloc(nt * sizeof(thread_info_t));

    // Creamos un array de threads.
    for (int i = 0; i < nt; i++) {
        // Inicializamos la estructura de cada thread.
        pool[i].num_thread = i;

        pthread_create(&pool[i].id,         // Identificador del thread.
                        NULL,               // Atributos por defecto. 0 == NULL.
                        haz_thread,         // Funcion de inicio del thread.
                        (void *) &pool[i]); // Argumentos para la funcion del thread.
    }

    for (int i = 0; i < nt; i++) {
        // Sincronizamos los threads.
        pthread_join(pool[i].id,
                    NULL);
        printf("Thread %i termino\n", pool[i].num_thread);
    }

    free(pool); // Liberamos la memoria reservada.

    return 0;
}

### Ejercicio 2
Modificar el programa anterior para que todos los threads esperen el tiempo suficiente para consultar sus identificadores en el sistema. Ejecutar el programa con 4 threads y completar la siguiente tabla con el comando ps, usar las opciones -L (mostrar threads) y la opción de formato -o con los campos adecuados. Identificar el thread principal en la tabla.

| PID | TID | TGID | PGID | CMD |
|---|---|---|---|---|
| 1918 | 1918 | 1918 | 1918 | ./ej1 6 |
| 1918 | 1920 | 1918 | 1918 | ./ej1 6 |
| 1918 | 1921 | 1918 | 1918 | ./ej1 6 |
| 1918 | 1924 | 1918 | 1918 | ./ej1 6 |
| 1918 | 1923 | 1918 | 1918 | ./ej1 6 |
| 1918 | 1924 | 1918 | 1918 | ./ej1 6 |
| 1941 | 1941 | 1941 | 1941 | ps -aL -o pid,tid... |

### Ejercicio 3
Escribir un programa que realice la suma paralela de los N primeros números naturales. Los argumentos del programa fijarán:
- Primer argumento, el número de threads (Nt).
- Segundo argumento, el tamaño de bloque (Tb) que determina cuántos números sumará cada thread.

Cada thread se le asignará un identificador (i = 0,1,2…) y sumará los enteros en el rango [Tb⋅i - (i+1)⋅Tb - 1] que agregará en una variable compartida, suma. El thread principal sincronizará todos los threads y mostrará la suma.


In [ ]:
#include <stdio.h>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>

// Definicion de la estructura para pasar argumentos a los threads.
typedef struct _thread_info_t{ // typedef sirve para definir un nuevo tipo de dato.
    int num_thread; // Numero de thread.
    int tb; // Tamaño de bloque.

    pthread_t id; // Identificador del thread a nivel de POSIX.
} thread_info;

typedef struct _suma_p_t{
    pthread_mutex_t mutex; // Mutex para proteger la variable compartida.
    int suma; // Variable compartida para la suma.
} suma_p_t;

// Variable global para la suma paralela.
suma_p_t suma = {PTHREAD_MUTEX_INITIALIZER, 0}; // Inicializamos el mutex y la suma a 0.

void* haz_thread(void* _info) {
    thread_info_t* info = (thread_info_t*) _info; // casteamos el puntero void a thread_info_t.
    
    int suma_parcial = 0;
    int ini = info->tb * info->num_thread;
    int fin = info->tb * (info->num_thread +1);

    printf("[%i] Sumando %i - %i\n", info->num_thread, ini, fin);
    for (int i = ini; i < fin; i++) {
        suma_parcial += i;
    }


    //  ESTO ES REGION CRITICA!!!!!!!!!!
    // Sumamos la suma parcial a la variable compartida.
    suma.suma += suma_parcial;
    // SE ACABA REGION CRITICA!!!!!!!!!!


    sleep(info->num_thread); //indica el numero de thread

    return NULL;
}

int main(int argc, char *argv[]) {
    // Primer argumento es el numero de threads.
    int nt = atoi(argv[1]);
    // Segundo argumento es el tamaño de bloque.
    int tb = atoi(argv[2]);

    // reservamos memoria para los threads "malloc(nt * sizeof(thread_info_t))".
    // Creamos un array de estructuras thread_info_t.
    thread_info_t * pool = (thread_info_t*) malloc(nt * sizeof(thread_info_t));

    // Creamos un array de threads.
    for (int i = 0; i < nt; i++) {
        // Inicializamos la estructura de cada thread.
        pool[i].num_thread = i;

        pthread_create(&pool[i].id,         // Identificador del thread.
                        NULL,               // Atributos por defecto. 0 == NULL.
                        haz_thread,         // Funcion de inicio del thread.
                        (void *) &pool[i]); // Argumentos para la funcion del thread.
    }

    for (int i = 0; i < nt; i++) {
        // Sincronizamos los threads.
        pthread_join(pool[i].id,
                    NULL);
        printf("Thread %i termino\n", pool[i].num_thread);
    }

    // esto es porque la suma de los n primeros numeros naturales es n*(n+1)/2 (suma de Gauss)
    int n = (nt * tb) - 1;
    printf("Suma total: %i / %i\n", suma.suma, (n *(n +1) /2 ));

    free(pool); // Liberamos la memoria reservada.

    return 0;
}

### Ejercicio 4
Escribir un programa con P threads productores (primer argumento) y C threads consumidores (segundo argumento) con las siguientes características:
- El productor escribirá en el buffer el elemento producido y el consumidor simplemente lo imprimirá en el terminal junto con las estadísticas del buffer (in, out y count). Los tiempos de producción y consumición serán de 1 y 2 segundos respectivamente.
- El buffer compartido será un array circular de tamaño fijo que estará representado por una estructura similar a la siguiente:

´´´
typedef struct _buffer
{
    int count;
    int in;
    int out;

    int data[BUFFER_SIZE];
} buffer_t;

´´´

Implementar el sistema con dos variables de condición que estarán asociadas a los siguientes  predicados: 
- Thread consumidor: “puedo consumir”, count > 0
- Thread producir: “puedo producir”,  count < BUFFER_SIZE 
- Cada productor producirá un número fijo de elementos (NUM_ELEMENTS).


In [ ]:
#include <stdio.h>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>

#define BUFFER_SIZE 10
#define P_ELEMENTS 5

// Definicion de la estructura para pasar argumentos a los threads.
typedef struct _thread_info_t{ // typedef sirve para definir un nuevo tipo de dato.
    int         num_thread; // Numero de thread.
    pthread_t   id;         // Identificador del thread a nivel de POSIX.
} thread_info_t;

typedef struct _buffer_t{
    pthread_mutex_t mutex; // Mutex para proteger el buffer.

    pthread_cond_t  produce; // Condicion para productores.
    pthread_cond_t  consume; // Condicion para consumidores.

    int in; // Indice de escritura.
    int out;// Indice de lectura.
    
    int elements; // Numero de elementos en el buffer.

    char data[BUFFER_SIZE]; // buffer de datos circular.
} buffer_t;

// Variable global para la suma paralela.
buffer_t buffer =   {PTHREAD_MUTEX_INITIALIZER, // Mutex para proteger el buffer.
                    PTHREAD_COND_INITIALIZER,   // Condicion para productores.
                    PTHREAD_COND_INITIALIZER,   // Condicion para consumidores.
                    0,                          // Indice de escritura.
                    0,                          // Indice de lectura. 
                    0};                         // Numero de elementos en el buffer.


void* productor(void* _info) {
    thread_info_t* info = (thread_info_t*) _info; // casteamos el puntero void a thread_info_t.

    for (int i = 0; i < NUM_ELEMENTS; i++) {
        // ENTRADA A REGION CRITICA.
        pthread_mutex_lock(&buffer.mutex); // Bloqueamos el mutex del buffer.
        
        // Esperamos mientras el buffer esté lleno.
        while(buffer.elements >= BUFFER_SIZE) {
            pthread_cond_wait(&buffer.produce,  // Espera en la condicion de produccion.
                                &buffer.mutex); // Desbloquea el mutex mientras espera.
        }

        int item = 100*info->num+i;

        // MODIFICACION DEL BUFFER.
        buffer.data[buffer.in] = item; // Producimos un dato.
        buffer.in = (buffer.in + 1) % BUFFER_SIZE; // Actualizamos el indice de escritura.
        buffer.elements++; // Incrementamos el numero de elementos en el buffer.

        printf("P[%i] E:%i I:%i in:%i out:%i\n", 
                info->num_thread, buffer.elements, item, 
                buffer.in, buffer.out);

        // SALIDA DE REGION CRITICA.
        pthread_cond_signal(&buffer.consume); // Señalizamos a los consumidores que hay datos.
        pthread_mutex_unlock(&buffer.mutex); // Desbloqueamos el mutex del buffer.

        slep(1); // Simulamos tiempo de produccion.
    }
    return NULL;
}

void* consumidor(void* _info) {
    thread_info_t* info = (thread_info_t*) _info; // casteamos el puntero void a thread_info_t.

    while (1) {
        // ENTRADA A REGION CRITICA.
        pthread_mutex_lock(&buffer.mutex); // Bloqueamos el mutex del buffer.
        
        // Esperamos mientras el buffer esté lleno.
        while(buffer.elements <= 0) {
            pthread_cond_wait(&buffer.consume,  // Espera en la condicion de consumir.
                                &buffer.mutex); // Desbloquea el mutex mientras espera.
        }

        // MODIFICACION DEL BUFFER.
        int item = buffer.data[buffer.out]; // Consumimos un dato.
        buffer.out = (buffer.out + 1) % BUFFER_SIZE; // Actualizamos el indice de lectura.
        buffer.elements--; // Reducimos el numero de elementos en el buffer.

        printf("C[%i] E:%i I:%i in:%i out:%i\n", 
                info->num_thread, buffer.elements, item, 
                buffer.in, buffer.out);

        // SALIDA DE REGION CRITICA.
        pthread_cond_signal(&buffer.produce); // Señalizamos a los productores que se ha consumido un dato.
        pthread_mutex_unlock(&buffer.mutex); // Desbloqueamos el mutex del buffer.

        sleep(1); // Simulamos tiempo de produccion.

    }

    return NULL;
}

int main(int argc, char *argv[]) {
    // Primer argumento es el numero de productores.
    int p = atoi(argv[1]);
    // Segundo argumento es el tamaño de consumidores.
    int c = atoi(argv[2]);

    // reservamos memoria para los threads "malloc(nt * sizeof(thread_info_t))".
    // Creamos un array de estructuras thread_info_t.
    thread_info_t * pool = (thread_info_t*) 
        malloc((p + c) * sizeof(thread_info_t));

    // Creamos los threads escritores.
    for (int i = 0; i < p; i++) {
        // Inicializamos la estructura de cada thread.
        pool[i].num_thread = i;

        pthread_create(&pool[i].id,         // Identificador del thread.
                        NULL,               // Atributos por defecto. 0 == NULL.
                        productor,          // Funcion de inicio del thread.
                        (void *) &pool[i]); // Argumentos para la funcion del thread.
    }

    // Creamos los threads lectores.
    for (int i = p; i < p + c; i++) {
        // Inicializamos la estructura de cada thread.
        pool[i].num_thread = i - p;

        pthread_create(&pool[i].id,         // Identificador del thread.
                        NULL,               // Atributos por defecto. 0 == NULL.
                        consumidor,           // Funcion de inicio del thread.
                        (void *) &pool[i]); // Argumentos para la funcion del thread.
    }

    for (int i = 0; i < (es + le); i++) {
        // Sincronizamos los threads.
        pthread_join(pool[i].id,
                    NULL);
        printf("Thread %i termino\n", pool[i].num_thread);
    }

    free(pool); // Liberamos la memoria reservada.

    return 0;
}

### Ejercicio 5
Añadir una condición de finalización al ejercicio 4 usando el patŕon centinela o píldora envenenada, de la siguiente forma:
- El thread principal sincronizará la ejecución de todos los productores para asegurarse de que todos los elementos se han producido.
- A continuación escribirá los elementos (píldoras envenenadas, por ejemplo valor -1) en el buffer, tantos como consumidores se hayan creado. Nota: Debe preservar el acceso concurrente al búfer, igual que el thread productor.
- Cuando un consumidor lea la píldora envenenada del búfer terminará después de ajustar los índices y señalizar la variable de condición.

In [ ]:
#include <stdio.h>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>


#define BUFFER_SIZE 10
#define NUM_ELEMENTOS_A_PRODUCIR 3

typedef struct _thread_info_t
{
	int num; // ID nuestro.
	pthread_t id; // ID de POSIX.
} thread_info_t;

typedef struct _buffer_t
{
	pthread_mutex_t mutex;

	// Variables de condicion, tantas como predicados haya.
	pthread_cond_t produce; // Si se puede producir o no. (elements < buffer_size).
	pthread_cond_t consume; // Si se puede consumir o no. (elements > 0).

	int data[BUFFER_SIZE];

	int in;
	int out;

	int nElements;
} buffer_t;

typedef struct _shared_int_t
{
	pthread_rwlock_t mutex; // Mutex para proteger el valor.
	int valor;
} shared_int_t;

buffer_t buffer = { PTHREAD_MUTEX_INITIALIZER, // Mutex.
			PTHREAD_COND_INITIALIZER, // Produce.
			PTHREAD_COND_INITIALIZER, // Consume.
			0, // In.
			0, // Out.
			0 }; // nElements.

void* productor(void *_info)
{
	thread_info_t *info = (thread_info_t*) _info; // Casting para poder utilizarlo.

	for(int i = 0; i < NUM_ELEMENTOS_A_PRODUCIR; ++i)
	{
		pthread_mutex_lock(&buffer.mutex); // Entrar a la region critica.
		while(buffer.nElements == BUFFER_SIZE) // Normalmente para esperar se pone la condicion negada.
		{
			pthread_cond_wait(&buffer.produce, &buffer.mutex); // Esperar.
		}
		// Modificacion del buffer.
		int item = 100 * info->num + i;
		buffer.data[buffer.in] = item;
		buffer.in = (buffer.in + 1) % BUFFER_SIZE; // Buffer circular.

		buffer.nElements++;

		printf("Pro[%i] nEl: %i Item: %i in: %i out: %i\n",
			info->num, buffer.nElements, item, buffer.in, buffer.out);

		pthread_cond_signal(&buffer.consume); // Depertar al siguiente en consume.
		pthread_mutex_unlock(&buffer.mutex);  // Desbloquear el mutex.

		sleep(1); // Esperar para que no intente ir otra vez a por el mutex inmediatamente.
	}
	return NULL;
}

void* consumidor(void *_info)
{
	thread_info_t *info = (thread_info_t*) _info; // Casting para poder utilizarlo.

	while(1)
	{
		pthread_mutex_lock(&buffer.mutex); // Entrar a la region critica.
		while(buffer.nElements == 0) // Al reves y suponiendo que no pueden ser < 0.
		{
			pthread_cond_wait(&buffer.consume, &buffer.mutex); // Esperar.
		}
		int item = buffer.data[buffer.out];
		buffer.out = (buffer.out + 1) % BUFFER_SIZE; // Buffer circular.

		buffer.nElements--;

		printf("Con[%i] nEl: %i Item: %i in: %i out: %i\n",
			info->num, buffer.nElements, item, buffer.in, buffer.out);

		pthread_cond_signal(&buffer.produce); // Despertamos al siguiente que produce.
		pthread_mutex_unlock(&buffer.mutex); // Desbloqueamos el mutex.

		if(item == -1) // Pildora envenada para acabar la consumicion. Valor a elegir.
		{
			break;
		}

		sleep(2); // Esperar para que no intente ir otra vez a por el mutex inmediatamente.
	}
	return NULL;
}

int main(int argc, char *argv[])
{
	int productores = atoi(argv[1]);
	int consumidores = atoi(argv[2]);

	thread_info_t *pool = (thread_info_t*)  malloc((productores + consumidores) * sizeof(thread_info_t)); 

	// Creacion de producotres.	
	for(int i = 0; i < productores; ++i)
	{
		pool[i].num = i;
		pthread_create(&pool[i].id, NULL, productor, (void*) &pool[i]);
	}

	// Creacion de consumidores.
	for(int i = productores; i < (productores + consumidores); ++i)
	{
		pool[i].num = i - productores;
		pthread_create(&pool[i].id, NULL, consumidor, (void*) &pool[i]);
	}

	// Escribir pildoras. Una por consumidor. Lo hace el thread principal.
	// Consumidores.
	for(int i = 0; i < consumidores; ++i)
	{
        pthread_mutex_lock(&buffer.mutex); // Entrar a la region critica.
		while(buffer.nElements == BUFFER_SIZE) // Normalmente para esperar se pone la condicion negada.
		{
			pthread_cond_wait(&buffer.produce, &buffer.mutex); // Esperar.
		}
        printf("hola");
		buffer.data[buffer.in] = -1; // Meter pildora.
		buffer.in = (buffer.in + 1) % BUFFER_SIZE; // Buffer circular.

		buffer.nElements++;

		pthread_cond_signal(&buffer.consume); // Depertar al siguiente en consume.
		pthread_mutex_unlock(&buffer.mutex); // Desbloquear el mutex. ???
	}

	// Sincronizar los threads.
	for(int i = 0; i < (productores + consumidores); ++i)
	{

		pthread_join(pool[i].id, NULL); // NULL porque no esperamos nada de retorno.
		printf("Thread %i termino\n", pool[i].num);
	}

	free(pool); // Liberar memoria.
	return 0;
}

El comportamiento esperado es que se ejecuten los consumidores, una vez ejecutados se envenenaran 
>buffer.data[buffer.in] = -1;

Y se queda colgado porque esto es una condicion de carrera, los productores esperan a que se consuma pero ya no se consume mas porque no hay consumidores.

### Ejercicio 6
Escribir un programa que cree L lectores (primer argumento) y E escritores (segundo argumento), de forma que:
- Los threads Lector imprimirán por pantalla un entero compartido y esperarán 0.1s con la llamada usleep(3). Este acceso lo repetirán 5 veces.
- Los threads Escritor incrementarán en 1 la variable y esperarán 0.25s. Este acceso lo repetirán 3 veces.
- El thread principal arrancará primero los escritores.

![image.png](attachment:image.png)

observacion: Posix prioriza los lectores a los escritores.


### Ejercicio 8

Utilizando como base el programa del ejercicio 4, diseñar e implementar una aplicación concurrente en C que explore un directorio del sistema de ficheros y compute el tamaño total de los ficheros regulares encontrado.

La aplicación aceptará dos argumentos, el primero será el directorio del que se quiere explorar; y el segundo será el número de threads Consumidores. Los threads de la aplicación implementarán la siguiente lógica:
> Thread productor
- Abre el directorio indicado y recorre sus entradas usando opendir() y readdir().
- Inserta las rutas completas de los ficheros o subdirectorios en un búfer circular compartido (usar snprintf())
> Threads consumidores
- Extraen rutas del búfer. Además la llamada al sistema para calcular el tamaño no debe mantener bloqueado el búfer compartido, así que el thread copiará la ruta en una variable local con strcpy().
- Una vez realizada la copia, actualizados los índices del búfer y  señalizada la variable de condición usará lstat() para obtener información del fichero. Si es un fichero regular acumulará su tamaño en una variable global protegida por otro mutex.
- Si el elemento es una “píldora venenosa” (cadena vacía) finalizará su ejecución.
> Thread principal
- Creará el thread productor y los threads consumidores.
- Cuando el productor termina de recorrer el directorio (pthread_join), inserta una píldora venenosa por cada consumidor para indicar el fin del trabajo.
- Finalmente espera (pthread_join) la terminación de todos los consumidores y muestra el tamaño total acumulado de los ficheros procesados.

Comprobar el funcionamiento del programa con diferentes niveles de concurrencia y la orden time(1) para evaluar el tiempo de ejecución en cada caso. Se pueden utilizar estructuras similares a las siguientes para gestionar los datos del programa

In [ ]:
#include <stdio.h>
#include <pthread.h>
#include <unistd.h>
#include <stdlib.h>
#include <string.h>
#include <sys/types.h>
#include <sys/stat.h>
#include <dirent.h>

#define BUFFER_SIZE 5

typedef struct thread_info_t
{
    int num;
    pthread_t id;

} thread_info_t;

typedef struct _buffer_t {

    pthread_mutex_t mutex;

    // elements < BUFFER_SIZE
    pthread_cond_t produce; // los consumidores cuando dejen espacio en el buffer le diran a los productores: "Produce!"
    // elements > 0
    pthread_cond_t consume; // los productores cuando dejen espacio en el buffer le diran a los consumidores: "Consume!"

    int in;
    int out;

    int elements;

    char data[BUFFER_SIZE][PATH_MAX];

} _buffer_t;
// --- EJERCICIO 8 ---
typedef struct _suma_t{
    pthread_mutex_t mutex;
    size_t          total;
}suma_t


suma_t suma = { PTHREAD_MUTEX_INITIALIZER,
                0};


_buffer_t buffer = {PTHREAD_MUTEX_INITIALIZER,  // mutex
                    PTHREAD_COND_INITIALIZER,   // produce
                    PTHREAD_COND_INITIALIZER,   // consume
                    0,                          // in
                    0,                          // out
                    0};                         // elements

void escribe_buffer(int t, int id, const char * item){
        pthread_mutex_lock(&buffer.mutex); // asegurarse que elements solo lo estoy modificando yo -> entramos en una region critica

        // buffer.elements > 0 -> predicado
        while(buffer.elements == 0) // la condicion se escribe justo al reves que el predicado
        {
            // esperar mientras no haya espacio en el buffer
            pthread_cond_wait(  &buffer.consume,
                                &buffer.mutex);
        }

        strcpy(buffer.data[buffer.in], item);

        buffer.in = (buffer.in + 1) % BUFFER_SIZE; // avanzamos en uno el puntero de lectura

        buffer.elements++;

        printf("C[%i] E:%i I:%i in:%i out:%i\n",
        t, id, buffer.elements, item, buffer.in, buffer.out);

        pthread_cond_signal(&buffer.consume); // hemos dejado un hueco en el buffer -> se puede producir
        pthread_mutex_unlock(&buffer.mutex);
    
}

void * productor(void * _path)
{
    char * path = (char *) _path; // casting al tipo
    
    char item[PATH_MAX];

    DIR * dir = opendir(path);
    // Variable con informacion del directorio.
    struct dirent * dent;
    // Mientras la lectura no de nulo escribimos el buffer.
    while((dent = readdir(dir))!= NULL){
        // Print de string
        sprintf(item, "%s/%s", 
                    path,
                    dent->d_name);// Nombre del fichero (NO ES EL PATH)
        // Escribimos en el buffer.
        escribe_buffer('P', 0, item);
    }

    return NULL;
}

void * consumidor(void * _info)
{
    thread_info_t * info = (thread_info_t*) _info; // casting al tipo

    // Cadena para copiar lo que hay en el buffer.
    char item[PATH_MAX];
    struct stat_ statbuf;

    while(1)
    { 
        pthread_mutex_lock(&buffer.mutex); // asegurarse que elements solo lo estoy modificando yo -> entramos en una region critica

        // buffer.elements > 0 -> predicado
        while(buffer.elements == 0) // la condicion se escribe justo al reves que el predicado
        {
            // esperar mientras no haya espacio en el buffer
            pthread_cond_wait(&buffer.consume, &buffer.mutex);
        }
        
        // Copiamos el elemento del buffer.
        strcpy(item, buffer.data[buffer.out]);
        buffer.out = (buffer.out + 1) % BUFFER_SIZE; // avanzamos en uno el puntero de lectura

        buffer.elements--;

        printf("C[%i] E:%i I:%i in:%i out:%i\n",
        info->num, buffer.elements, item, buffer.in, buffer.out);

        pthread_cond_signal(&buffer.produce); // hemos dejado un hueco en el buffer -> se puede producir
        pthread_mutex_unlock(&buffer.mutex);
           
        // POISON PILL
        if (item == '\0') // si el primer elemento es vacio, es final de cadena.
        {
            break;
        }

        sleep(2);
    }
    // Devuelveme el inodo asociado a la ruta item.
    stat(item, &statbuf);
    
    if(S_ISREG(statbuf.st_mode)){
        printf("c[%i]: %s -%i\n", info->num,
                                    item, 
                                    statbuf.st_size);
        
        pthread_mutex_lock(&suma.mutex);
        suma.total += statbuf.st_size;
        pthread_mutex_unlock(&suma.mutex);
    }

    return NULL;
}

void main(int argc, char *argv[])
{
    int c = atoi(argv[2]);

    thread_info_t * pool = (thread_info_t *) malloc((p+c)*sizeof(thread_info_t));

    pthread_t productor;
    pthread_create(&productor,
                    NULL, 
                    productor,
                    (void *) argv[1]);

    for(int i = 0; i < c; ++i) 
    {
        pool[i].num = i - 1;
        pthread_create( &(pool[i].id),
                        NULL, 
                        consumidor,
                        (void*) &pool[i]);
    }
    pthread_join(productor, NULL);
    // Escribimos las pildoras
    for(int i = 0; i < c; ++i)
    {
        escribe_buffer("M", 0, "");
    } 

    for(int i = 0; i < c; ++i)
    {
        pthread_join(pool[i].id, NULL);
        printf("Thread %i termino\n", pool[i].num);
    }

    printf("Tamaño total: %i", suma.total);

    free(pool);

    return 0;
}